# LangChain 表达式语言
- 一种在langchian之上封装的高级解释语言
- 简化链条开发，支撑真实生产环境而发明
- 更好的流式支持
- 轻松获取中间步骤
- 更好的异步支持
- 输入输出强验证
- 优化执行时间
- 支持重试和反馈
- 无缝追踪集成
- 无缝部署集成

# Runnable接口
- stream  流式返回响应接口
- invoke 给chain一个输入
- batch 给chain多个输入
- astream 以异步方式返回数据
- ainvoke 异步方式给chain输入
- abatch 异步方式给chain多个输入
- Astream_log 实时将中间步骤流式传输

In [1]:
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv()

True

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("给我讲一个关于 {topic}的笑话")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "冰激凌"})

'为什么冰激凌会变得那么软？因为它总是在被批评，所以变得融化了！哈哈哈！'

In [4]:
prompt_value = prompt.invoke({"topic": "刺猬"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='给我讲一个关于 刺猬的笑话')])

In [5]:
prompt_value.to_messages()

[HumanMessage(content='给我讲一个关于 刺猬的笑话')]

In [6]:
prompt_value.to_string()

'Human: 给我讲一个关于 刺猬的笑话'

In [7]:
message = model.invoke(prompt_value)
message

AIMessage(content='为什么刺猬永远不会迟到？\n\n因为他总是能及时“刺”到目的地！😄😄😄', response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 22, 'total_tokens': 64, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-427abfbf-9798-4f7a-9b67-935a569a3676-0')

# 使用llm的区别

In [9]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")
llm.invoke(prompt_value)

AIMessage(content='为什么刺猬不喜欢和别人分享食物呢？\n\n因为他们觉得吃了之后会“刺”倒别人！哈哈哈！', response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 22, 'total_tokens': 72, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5740175b-39bd-41df-8f18-b5a302e3ce9e-0')

In [10]:
output_parser.invoke(message)

'为什么刺猬永远不会迟到？\n\n因为他总是能及时“刺”到目的地！😄😄😄'

# RAG Search Exampl
- 建立向量数据
- 使用RAG增强

In [11]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [12]:
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [13]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [14]:
chain.invoke({"question": "where did harrison work", "language": "chinese"})

'Harrison 在 Kensho 工作。'

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("给我讲一个关于 {topic}的笑话")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "冰激凌"})

'为什么冰激凌会笑？\n因为它被人吃了，哈哈哈！'

In [3]:
# chain 的输入要求是什么
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [4]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [5]:
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {

In [6]:
chain.output_schema.schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}

# 流式

In [10]:
for s in chain.stream({"topic": "女声"}):
    print(s, end="", flush=True)

为什么女声笑起来总是那么动听？因为她们有个“开心”的开关，一旦按下就会笑个不停！

In [11]:
chain.invoke({"topic": "熊"})

'好的，这里有一个关于熊的笑话：\n\n为什么熊不喜欢下雨天？\n\n因为它们怕会被“熊”溼（淋湿）！哈哈哈！😄。'

# Batch

In [12]:
chain.batch([{"topic": "熊"}, {"topic": "猫"}])

['好的，这里有一个熊的笑话：为什么熊不喜欢下雨天？因为它们怕淋湿了会变成“熊猫”！哈哈哈！希望你喜欢这个笑话！',
 '为什么猫喜欢打麻将？\n\n因为它们喜欢抓牌！哈哈哈！']

In [13]:
#max_concurrency控制并发数
chain.batch([{"topic": "熊"}, {"topic": "猫"}, {"topic": "狗"}], config={"max_concurrency": 5})

['为什么熊喜欢在森林里开派对？\n因为那里有很多树林啊！',
 '当猫问另一只猫：“你为什么总是这么干净？” 另一只猫回答：“因为我每天都在梳理我的‘美’发！”',
 '当狗站在街边，看着经过的汽车时，它会为什么摇着尾巴？\n\n因为它知道汽车是“汪汪”开的！😄🐶']

# 异步流式

In [14]:
async for s in chain.astream({"topic": "女人"}):
    print(s, end="", flush=True)

好的，这里有一个关于女人的笑话：

为什么女人总是迟到？

因为她们每次都觉得自己还可以再花点时间变得更完美！哈哈哈。

# Async Invoke

In [15]:
await chain.ainvoke({"topic": "男人"})

'好的，这里有一个关于男人的笑话：\n\n为什么男人总是忘记关灯？\n\n因为他们总是期待有女人来帮他们关灯。'

# Async Batch

In [16]:
await chain.abatch([{"topic": "熊"},{"topic": "女人"}])

['好的，这里有一个关于熊的笑话：\n\n为什么熊不喜欢下雨天？\n\n因为它们怕变成“湿熊”啦！哈哈哈！😄',
 '好的，这里有一个关于女人的笑话：\n\n为什么女人比男人聪明？因为女人知道如何在一分钟内改变七种心情，而男人却需要七分钟才能弄清楚到底发生了什么事情。哈哈哈！希望你喜欢这个笑话！😄😄😄']

# 异步获取中间步骤


In [17]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """基于下面的上下文来回答问题:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["柯基犬是一种中型家庭宠物犬"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [18]:
async for chunk in retrieval_chain.astream_log(
    "柯基是什么?", include_names=["Docs"]
):
    print("-" * 40)
    print(chunk)

----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '2cd3562c-e987-41ee-8b3a-1385b698af0c',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': 'f6166767-4985-4681-ac25-15963d97b2e3',
            'metadata': {},
            'name': 'Docs',
            'start_time': '2024-05-11T08:49:59.016+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['map:key:context', 'FAISS', 'OpenAIEmbeddings'],
            'type': 'retriever'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs/final_output',
  'value': {'documents': [Document(page_content='柯基犬是一种中型家庭宠物犬')]}},
 {'op': 'add'

# 只看状态值

In [19]:
async for chunk in retrieval_chain.astream_log(
    "柯基是什么?", include_names=["Docs"], diff=False
):
    print("-" * 70)
    print(chunk)

----------------------------------------------------------------------
RunLog({'final_output': None,
 'id': '1dd39efb-4247-4ca0-a7a7-536aa7033d8d',
 'logs': {},
 'name': 'RunnableSequence',
 'streamed_output': [],
 'type': 'chain'})
----------------------------------------------------------------------
RunLog({'final_output': None,
 'id': '1dd39efb-4247-4ca0-a7a7-536aa7033d8d',
 'logs': {'Docs': {'end_time': None,
                   'final_output': None,
                   'id': '2951922e-0da1-471b-9c0a-e2c7154eaa80',
                   'metadata': {},
                   'name': 'Docs',
                   'start_time': '2024-05-11T08:50:59.150+00:00',
                   'streamed_output': [],
                   'streamed_output_str': [],
                   'tags': ['map:key:context', 'FAISS', 'OpenAIEmbeddings'],
                   'type': 'retriever'}},
 'name': 'RunnableSequence',
 'streamed_output': [],
 'type': 'chain'})
-------------------------------------------------------------

# 并行支持

In [20]:
from langchain_core.runnables import RunnableParallel

chain1 = ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话") | model
chain2 = (
    ChatPromptTemplate.from_template("写两行关于{topic}的诗歌")
    | model
)
combined = RunnableParallel(joke=chain1, poem=chain2)

In [32]:
%%time
chain1.invoke({"topic": "熊"})

CPU times: total: 15.6 ms
Wall time: 1.41 s


AIMessage(content='为什么熊不喜欢下雨？因为它们怕变成“湿熊”！😄', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 20, 'total_tokens': 55, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bb67c581-0a63-4622-9c84-23b1b32579d3-0')

In [34]:
%%time
chain2.invoke({"topic": "代码"})

CPU times: total: 0 ns
Wall time: 1.12 s


AIMessage(content='In a world of symbols and syntax divine,\nProgrammers craft beauty line by line.', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 18, 'total_tokens': 35, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4b051f35-6cd9-40ce-8d6c-c4e49fe777ad-0')

# 并行

In [35]:
%%time
combined.invoke({"topic": "熊"})

CPU times: total: 31.2 ms
Wall time: 1.43 s


{'joke': AIMessage(content='为什么熊喜欢在树林里睡觉？\n\n因为那里有“枕木”可以用！哈哈哈！', response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 20, 'total_tokens': 60, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-85b0f443-3d73-44a4-95d4-b7a3d288bd68-0'),
 'poem': AIMessage(content='森林深处觅食忙，熊儿挥爪捕鱼香。', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 20, 'total_tokens': 52, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ab5247ae-f09e-4415-b191-a4ec7bf38c89-0')}

# 并行批处理，适用于大量生成

In [36]:
%%time
chain1.batch([{"topic": "熊"}, {"topic": "猫"}])

CPU times: total: 31.2 ms
Wall time: 2.63 s


[AIMessage(content='为什么熊不喜欢雨天？因为它们会变成“湿熊”！哈哈哈！😆😆😆', response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 20, 'total_tokens': 62, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3a6f0b4c-a14a-475c-a2ba-2150098893bf-0'),
 AIMessage(content='为什么猫不喜欢下雨天？\n因为它们不想变成“湿猫”！哈哈哈！😸', response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 20, 'total_tokens': 61, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-276a29fd-3c4c-4ae4-8a99-e0cb59e7b3e9-0')]

In [30]:
%%time
chain2.batch([{"topic": "熊"}, {"topic": "猫"}])

CPU times: total: 0 ns
Wall time: 1.87 s


[AIMessage(content='森林深处熊探头，蜂蜜香甜引诱游。\n毛茸茸身影慵懒睡，威武霸气挡路愁。', response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 20, 'total_tokens': 81, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c5743e76-37be-497c-b02a-ee50a56384f0-0'),
 AIMessage(content='猫儿悠闲卧在窗前，\n嘴角微微上扬，慵懒又神秘。', response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 20, 'total_tokens': 61, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f9f64eb1-aa5a-4091-94cd-753eed8abc8f-0')]

In [38]:
%%time
combined.batch([{"topic": "熊"}, {"topic": "猫"},{"topic": "狗"},{"topic": "鸭"}])

CPU times: total: 62.5 ms
Wall time: 2.86 s


[{'joke': AIMessage(content='为什么熊不喜欢下雨天？\n因为它们怕被泡成熊猫！😄😄😄', response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 20, 'total_tokens': 62, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dafad4d8-79e5-43ea-97c4-f2d5bb0ca614-0'),
  'poem': AIMessage(content='森林深处有只熊，毛茸茸身姿威武雄。\n它悠闲地漫步山间，享受着大自然的恩宠。', response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 20, 'total_tokens': 78, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f4dc43a7-ba7f-49ff-9dbd-bb1162890f5e-0')},
 {'joke': AIMessage(content='为什么猫喜欢打猎？因为他们喜欢“捉”住机会！😸😹', response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 20, 'total_tokens': 57, 'pre_token_count': 16384, 'pre_total'